In [1]:
import pymongo
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC, SVR
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer 
from sklearn.metrics import r2_score

In [2]:
import nltk
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer

import string
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mariam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mariam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\mariam\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mariam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
client = MongoClient()
#point the client at mongo URI
client = MongoClient('localhost' , 27017)
#select database
db = client['DB_course']
#select the collection within the database
udemy = db.udemy
#convert entire collection to Pandas dataframe
udemyData = pd.DataFrame(list(udemy.find()))
# select the collection within the database
udacity = db.udacity
#convert entire collection to Pandas dataframe
udacityData = pd.DataFrame(list(udacity.find()))

In [4]:
udemyData.head(5)

,_id,title,id,price,link,description,instructors,source
0,61487cabdc1661e7e10dddab,CCNP ROUTE 300-101 Video Boot Camp With Chris ...,8420,$19.99,/course/ccnp-route-on-demand-video-boot-camp/,Pass The CCNP ROUTE 300-101 Exam With Chris Br...,[Chris],udemy
1,61487cacdc1661e7e10dddac,Become a Wine Connoisseur,8812,$24.99,/course/get-wine-savvy/,Learn about wine from vine to table from the e...,[Palate],udemy
2,61487cacdc1661e7e10dddad,Learn Serbian 102,9327,$99.99,/course/serbian-202/,"Perfect for false beginners in Serbian, Bosnia...",[Marina],udemy
3,61487cacdc1661e7e10dddae,Adobe Acrobat X Professional Tutorial - Learn ...,10320,$49.99,/course/learning-adobe-acrobat-x/,Master Acrobat X. Create a PDF then edit and s...,[Infinite],udemy
4,61487cacdc1661e7e10dddaf,Improve Your Physical and Mental Success with ...,21256,$39.99,/course/improve-your-physical-and-mental-succe...,Learn Tai Chi to improve your physical perform...,[Ramel],udemy


In [5]:
udacityData.tail(5)

,_id,title,instructors,level,description,prerequisites,price,source
237,61487ca101fe6c4547ac5b54,Introduction to Python,None,beginner,,,[],udacity
238,61487ca101fe6c4547ac5b55,Product Strategy,None,beginner,,,[],udacity
239,61487ca101fe6c4547ac5b56,Supervised Learning,None,beginner,,,[],udacity
240,61487ca101fe6c4547ac5b57,Intro to Data Analysis,None,beginner,,,[],udacity
241,61487ca101fe6c4547ac5b58,Unsupervised Learning,None,beginner,,,[],udacity


In [6]:
print(udacityData.shape)
print(udemyData.shape)

(242, 8)
(100, 8)


In [7]:
print(udacityData.isna().sum())

print("______________________________________________")

print(udemyData.isna().sum())

_id               0
title             0
instructors      31
level             0
description       0
prerequisites     0
price             0
source            0
dtype: int64
______________________________________________
_id            0
title          0
id             0
price          0
link           0
description    0
instructors    0
source         0
dtype: int64


In [8]:
udacityData.level.unique()

array(['advanced', 'intermediate', 'beginner', ''], dtype=object)

In [9]:
pricesPerLevel = { '' : '$25',
                'beginner' : '$75' ,
                 'intermediate' :'$100' ,
                 'advanced' : '$120'}
prices = []
for level in udacityData['level']:
    prices.append(pricesPerLevel[level])
udacityData['price'] = prices

In [10]:
udacityData.head(5)

,_id,title,instructors,level,description,prerequisites,price,source
0,61487c9301fe6c4547ac5a67,Artificial Intelligence for Robotics,[Sebastian Thrun],advanced,<p>Learn how to program all the major systems ...,<p>Success in this course requires some progra...,$120,udacity
1,61487c9401fe6c4547ac5a68,Applied Cryptography,[Dave Evans],advanced,"<p>Cryptography is present in everyday life, f...",<p>This course assumes previous programming ex...,$120,udacity
2,61487c9401fe6c4547ac5a69,Interactive 3D Graphics,"[Eric Haines, Gundega Dekena]",intermediate,<p>This class will teach you about the basic p...,<p><strong>Knowing how to program in some lang...,$100,udacity
3,61487c9401fe6c4547ac5a6a,Software Debugging,"[Andreas Zeller, Gundega Dekena]",intermediate,<p>In this class you will learn how to debug p...,<p>Basic knowledge of programming and Python a...,$100,udacity
4,61487c9401fe6c4547ac5a6b,Intro to Artificial Intelligence,"[Peter Norvig, Sebastian Thrun]",intermediate,<p>Artificial Intelligence (AI) is a field tha...,<p>Some of the topics in Introduction to Artif...,$100,udacity


In [11]:
df = pd.concat([udemyData, udacityData])

In [12]:
df

,_id,title,id,price,link,description,instructors,source,level,prerequisites
0,61487cabdc1661e7e10dddab,CCNP ROUTE 300-101 Video Boot Camp With Chris ...,8420.0,$19.99,/course/ccnp-route-on-demand-video-boot-camp/,Pass The CCNP ROUTE 300-101 Exam With Chris Br...,[Chris],udemy,NaN,NaN
1,61487cacdc1661e7e10dddac,Become a Wine Connoisseur,8812.0,$24.99,/course/get-wine-savvy/,Learn about wine from vine to table from the e...,[Palate],udemy,NaN,NaN
2,61487cacdc1661e7e10dddad,Learn Serbian 102,9327.0,$99.99,/course/serbian-202/,"Perfect for false beginners in Serbian, Bosnia...",[Marina],udemy,NaN,NaN
3,61487cacdc1661e7e10dddae,Adobe Acrobat X Professional Tutorial - Learn ...,10320.0,$49.99,/course/learning-adobe-acrobat-x/,Master Acrobat X. Create a PDF then edit and s...,[Infinite],udemy,NaN,NaN
4,61487cacdc1661e7e10dddaf,Improve Your Physical and Mental Success with ...,21256.0,$39.99,/course/improve-your-physical-and-mental-succe...,Learn Tai Chi to improve your physical perform...,[Ramel],udemy,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
237,61487ca101fe6c4547ac5b54,Introduction to Python,NaN,$75,NaN,,None,udacity,beginner,
238,61487ca101fe6c4547ac5b55,Product Strategy,NaN,$75,NaN,,None,udacity,beginner,
239,61487ca101fe6c4547ac5b56,Supervised Learning,NaN,$75,NaN,,None,udacity,beginner,
240,61487ca101fe6c4547ac5b57,Intro to Data Analysis,NaN,$75,NaN,,None,udacity,beginner,


In [13]:
df.isna().sum()

_id                0
title              0
id               242
price              0
link             242
description        0
instructors       31
source             0
level            100
prerequisites    100
dtype: int64

In [14]:
df.drop(['id','_id','link','prerequisites','instructors','description'], inplace = True , axis = 1)

In [15]:
df

,title,price,source,level
0,CCNP ROUTE 300-101 Video Boot Camp With Chris ...,$19.99,udemy,NaN
1,Become a Wine Connoisseur,$24.99,udemy,NaN
2,Learn Serbian 102,$99.99,udemy,NaN
3,Adobe Acrobat X Professional Tutorial - Learn ...,$49.99,udemy,NaN
4,Improve Your Physical and Mental Success with ...,$39.99,udemy,NaN
...,...,...,...,...
237,Introduction to Python,$75,udacity,beginner
238,Product Strategy,$75,udacity,beginner
239,Supervised Learning,$75,udacity,beginner
240,Intro to Data Analysis,$75,udacity,beginner


In [16]:
df['source'] = df['source'].replace({'udemy':1, 'udacity':2})
df

,title,price,source,level
0,CCNP ROUTE 300-101 Video Boot Camp With Chris ...,$19.99,1,NaN
1,Become a Wine Connoisseur,$24.99,1,NaN
2,Learn Serbian 102,$99.99,1,NaN
3,Adobe Acrobat X Professional Tutorial - Learn ...,$49.99,1,NaN
4,Improve Your Physical and Mental Success with ...,$39.99,1,NaN
...,...,...,...,...
237,Introduction to Python,$75,2,beginner
238,Product Strategy,$75,2,beginner
239,Supervised Learning,$75,2,beginner
240,Intro to Data Analysis,$75,2,beginner


In [17]:
df['level'] = df['level'].fillna(0)
df

,title,price,source,level
0,CCNP ROUTE 300-101 Video Boot Camp With Chris ...,$19.99,1,0
1,Become a Wine Connoisseur,$24.99,1,0
2,Learn Serbian 102,$99.99,1,0
3,Adobe Acrobat X Professional Tutorial - Learn ...,$49.99,1,0
4,Improve Your Physical and Mental Success with ...,$39.99,1,0
...,...,...,...,...
237,Introduction to Python,$75,2,beginner
238,Product Strategy,$75,2,beginner
239,Supervised Learning,$75,2,beginner
240,Intro to Data Analysis,$75,2,beginner


In [18]:
df['level'] = df['level'].replace({0:0, 'beginner':1, 'intermediate':2, 'advanced':3,"":4})
df

,title,price,source,level
0,CCNP ROUTE 300-101 Video Boot Camp With Chris ...,$19.99,1,0
1,Become a Wine Connoisseur,$24.99,1,0
2,Learn Serbian 102,$99.99,1,0
3,Adobe Acrobat X Professional Tutorial - Learn ...,$49.99,1,0
4,Improve Your Physical and Mental Success with ...,$39.99,1,0
...,...,...,...,...
237,Introduction to Python,$75,2,1
238,Product Strategy,$75,2,1
239,Supervised Learning,$75,2,1
240,Intro to Data Analysis,$75,2,1


In [19]:
df['level'].unique()

array([0, 3, 2, 1, 4], dtype=int64)

In [20]:
df.isna().sum()

title     0
price     0
source    0
level     0
dtype: int64

In [21]:
pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', -1)

C:\Users\mariam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [22]:
df['title']

0      CCNP ROUTE 300-101 Video Boot Camp With Chris Bryant        
1      Become a Wine Connoisseur                                   
2      Learn Serbian 102                                           
3      Adobe Acrobat X Professional Tutorial - Learn The Easy Way  
4      Improve Your Physical and Mental Success with Tai Chi       
5      How to Be a Commercial Photographer                         
6      AJAX Development                                            
7      Actionscript 3 (Beginner to Advanced) + Mobile Games        
8      WordPress Essentials                                        
9      Plein Air Essentials - Learn Basics of Plein Air Painting   
10     Building Your Platform                                      
11     Powerful Business Writing #2 – How to Write in Fewer Words  
12     IP Subnetting Masterclass - Learn the Easy Way to Subnet    
13     DJ Course in Traktor & Virtual DJ                           
14     Real Pilates 7 Day Core Strength & Stamin

In [23]:
stopwords = nltk.corpus.stopwords.words('english')
words = set(nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()

In [24]:
df['title']= df['title'].str.replace(r'<[^<>]*>', '')
df['title']= df['title'].str.replace(r':','')
df['title']= df['title'].str.replace('[^a-zA-Z]',' ')

df['price'] = df['price'].str.replace(r'$','')
df['price'] = df['price'].astype(float)

In [25]:
df['title']= df['title'].str.replace('[{}]'.format(string.punctuation), '')
df['title']= df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [26]:
df

,title,price,source,level
0,CCNP ROUTE Video Boot Camp With Chris Bryant,19.99,1,0
1,Become Wine Connoisseur,24.99,1,0
2,Learn Serbian,99.99,1,0
3,Adobe Acrobat X Professional Tutorial Learn The Easy Way,49.99,1,0
4,Improve Your Physical Mental Success Tai Chi,39.99,1,0
5,How Be Commercial Photographer,89.99,1,0
6,AJAX Development,19.99,1,0
7,Actionscript Beginner Advanced Mobile Games,49.99,1,0
8,WordPress Essentials,49.99,1,0
9,Plein Air Essentials Learn Basics Plein Air Painting,19.99,1,0


In [27]:
X = df[['source','level']] 

#the column text contains textual data to extract features from.
y = df['price']

#this is the column we are learning to predict.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [28]:
X['level'].unique()

array([0, 3, 2, 1, 4], dtype=int64)

In [29]:
'''
#Step 2-3: Pre-process and Vectorize train and test data
vect = CountVectorizer(max_features=500) 
#clean is a function we defined for pre-processing, seen in the notebook.
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
print(X_train_dtm.shape, X_test_dtm.shape)
'''

'\n#Step 2-3: Pre-process and Vectorize train and test data\nvect = CountVectorizer(max_features=500) \n#clean is a function we defined for pre-processing, seen in the notebook.\nX_train_dtm = vect.fit_transform(X_train)\nX_test_dtm = vect.transform(X_test)\nprint(X_train_dtm.shape, X_test_dtm.shape)\n'

In [30]:
models = []

models.append(("SVR",SVR()))
models.append(("RandomForest",RandomForestRegressor()))

In [31]:
accuracy = []
names = []
for name,model in models:
    model.fit(X_train, y_train)
    y_pred_class = model.predict(X_test)    
    names.append(name)
    accuracy.append(r2_score(y_test, y_pred_class))

for i in range(len(names)):
    print("{} accuracy = {:.3f}".format(names[i],accuracy[i]))

SVR accuracy = 0.776
RandomForest accuracy = 0.785
